In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import io
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [3]:
sitemaps = parse_sitemap(io.StringIO(req.get('https://www.vesti.ru/sitemap.xml').content.decode()))
len(sitemaps)

32

In [4]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemaps):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-4-1b27f3d996d8>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemaps):


In [7]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True)
df.head()

,url,lastmod
0,https://www.vesti.ru/article/2498253,2020-12-13 14:53:59+00:00
1,https://www.vesti.ru/article/2498252,2020-12-13 14:54:09+00:00
2,https://www.vesti.ru/article/2498181,2020-12-13 14:48:15+00:00
3,https://www.vesti.ru/article/2498251,2020-12-13 14:54:53+00:00
4,https://www.vesti.ru/article/2498235,2020-12-13 14:32:25+00:00


In [8]:
df[(df.lastmod >= '2019-11-01')].to_json('news_vesti_db.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape

(149642, 2)